## 1.2 Dataset

In [ ]:
# !pip install glob2

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import image_dataset_from_directory
import numpy as np
# (x_train, _), (x_test, _) = tf.keras.utils.image_dataset_from_directory(

# Configurations principales de nos modèles
IMG_SIZE          = 448             # taille coté final d'une image en pixel (ici 28x28)
NB_EPOCHS_DENOISE = 10               # nombre epoch alogithme debruiter
BATCH_SIZE        = 8             # taille batch de traitement
SAV_MODEL_DENOISE = "denoiser.h5"     # sauvegarde du modele de debruitage

def process(image):
    image = tf.cast(image/255. ,tf.float32)
    return image


# Import du .env
import dotenv
import os

# Chargement du .env !!!!!!!!!!!! CHANGER LE PATH !!!!!!!!!!!!!!
# Renvoie true si le .env est chargé
dotenv.load_dotenv('/tf/science-de-la-donnee/.env')

SOURCE_LIVRABLE2_PATH = os.getenv("SOURCE_LIVRABLE2_PATH")
print(SOURCE_LIVRABLE2_PATH)

In [ ]:

x_train = image_dataset_from_directory(
    SOURCE_LIVRABLE2_PATH,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode="categorical",
    # label_mode=None,
    shuffle=False,
    validation_split=0.2,
    subset="training",
    seed=123,
    color_mode="rgb"
    )

x_test = image_dataset_from_directory(
    SOURCE_LIVRABLE2_PATH,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode="categorical",
    # label_mode=None,
    shuffle=False,
    validation_split=0.2,
    subset="validation",
    seed=123,
    color_mode="rgb"
    )

AUTOTUNE = tf.data.experimental.AUTOTUNE

x_train = x_train.map(lambda x,y: (x/255,y))
x_test = x_test.map(lambda x,y: (x/255,y))

In [ ]:
%ls ./photoOnly/Photo

Commençons par écrire une fonction qui permet de visualiser $n$ premiers enregistrements en noir et blanc. 

In [ ]:
import matplotlib.pyplot as plt

import os
import tensorflow as tf
# os.chdir(r'/tf')





def add_noise(img, noise_factor=25):
    noise = tf.random.normal(shape=tf.shape(img), mean=0.0, stddev=noise_factor/255, dtype=tf.float32)
    noised_img = tf.cast(img, tf.float32) + noise
    # noised_img = tf.clip_by_value(noised_img, clip_value_min=0.0, clip_value_max=1.0)
    # noised_img = noised_img /255
    # img = img /255
    return noised_img, img

#  divide by 255


x_train_noisy = x_train.map(lambda x,y: (add_noise(x, 40)))
x_test_noisy = x_test.map(lambda x,y: (add_noise(x, 40)))


def display_images(X, n):
    plt.figure(figsize=(7, 7))
    # print(X.shape)
    for i in range(n):
    #A COMPLETER
        # print(X[i].shape)
        ax = plt.subplot(1, n, i + 1)
                         #A COMPLETER
        plt.imshow(X[i])
                   #A COMPLETER
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()



In [ ]:

display_images(list(x_train_noisy.take(1).as_numpy_iterator())[0][0], n=1)
display_images(list(x_train.take(1).as_numpy_iterator())[0][0], n=1)

Voyons ce que ça donne :

# AutoEncoder

In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D


# The encoding process
input_img = Input(shape=(IMG_SIZE, IMG_SIZE, 3))  # adapt this if using `channels_first` image data format

# Encoding #

# Conv1 #
x = Conv2D(8, (3, 3), activation='relu', padding='same')(input_img)  # 16 filters, 3x3 kernel
x = MaxPooling2D((2, 2), padding='same')(x)  # Pooling with 2x2 grid

# Conv2 #
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)  # 8 filters, 3x3 kernel
encoded = MaxPooling2D((2, 2), padding='same')(x)  # Pooling with 2x2 grid


# Note:
# padding is a hyper-arameter for either 'valid' or 'same'. 
# "valid" means "no padding". 
# "same" results in padding the input such that the output has the same length as the original input.

###### Décodeur

Passon au décodeur. Écrivez un code qui permet de décoder une image encodée selon l'architecture du réseau de neurones décrite auparavent. 

In [ ]:
x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)  # 8 filters, 3x3 kernel
x = UpSampling2D((2, 2))(x)  # Upsampling to 2x2 grid
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)  # 16 filters, 3x3 kernel
x = UpSampling2D((2, 2))(x)  # Upsampling to 2x2 grid
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)  # 1 filter to match input, 3x3 kernel


In [ ]:
from tensorflow.keras.models import Model

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()


###### Entrainement de l'auto-encodeur

On va ensuite entraîner l'auto-encodeur en utilisant les constantes définit au début (`NB_EPOCHS_DENOISE,BATCH_SIZE`)

In [ ]:
%reload_ext tensorboard

Affichez maintenant la courbe d'apprentissage. Que pensez-vous des performances du modèle ?

In [ ]:
history = autoencoder.fit(
    x_train_noisy,
    epochs=NB_EPOCHS_DENOISE,
    shuffle=True,
    validation_data=x_test_noisy,
    # callbacks=[tf.keras.callbacks.TensorBoard(log_dir='./tmp/autoencoder')]
    )


In [ ]:
pred = autoencoder.predict(x_test_noisy)

In [ ]:
display_images(list(x_test_noisy.take(1).as_numpy_iterator())[0][0], n=1)
display_images(pred, n=1)

In [ ]:
# Visualisation des pertes d'apprentissage (Train) et de validation (Test)
plt.plot( #A COMPLETER
         label='train')
plt.plot( #A COMPLETER
         label='test')
plt.legend()

Que pensez-vous des performances du modèle ?
<em>À COMPLÉTER</em>


# 1.4 Sauvgarde de l'auto-encodeur

L'entrainement de l'auto-encodeur sans utilisation de puissance de calcul (GPU) peut prendre beaucoup de temps. Usuellement, nous sauvegardons le modèle entraîné en local ou sur un serveur distant pour l'utiliser ultérieurement afin de traiter les nouvelles données (d'ailleurs, vous verrez une utilisation avancée de cette technique, le transfert learning, dès la semaine prochaine).
Pour sauvegarder le modèle `autoencoder`, utiliser la méthode `save`.

In [ ]:
# save the model
#A COMPLETER


In [ ]:
decoded_imgs = #A COMPLETER
